# Generation Tasks: Performance by Layer

In [ ]:
import json
from pathlib import Path

import pandas as pd
from scipy.stats import hmean

RESULTS_ROOT = Path("../../results")


def load_json(file):
    with file.open("r") as handle:
        return json.load(handle)


def layer_dirs(results_dir):
    return sorted(results_dir.iterdir(), key=lambda x: int(x.name))


def load_gen_metrics(model, dataset):
    gen_results_dir = RESULTS_ROOT / f"emnlp_sweep_gen_{dataset}_{model}"
    assert gen_results_dir.exists()

    if dataset == "biosbias":
        gen_results_dir /= "decontextual"

    rows = []
    for gen_layer_dir in layer_dirs(gen_results_dir / "linear")[:-1]:
        layer = int(gen_layer_dir.name)
        if dataset == "counterfact":
            efficacy = load_json(gen_layer_dir / "efficacy_metrics.json")
            paraphrase = load_json(gen_layer_dir / "paraphrase_metrics.json")
            generation = load_json(gen_layer_dir / "generation_metrics.json")
            essence = load_json(gen_layer_dir / "essence_metrics.json")

            efficacy_score = paraphrase["score"]["mean"]
            fluency_score = generation["fluency"]["mean"]
            consistency_score = generation["consistency"]["mean"]
            essence_score = essence["essence"]["mean"]
            row = {
                "layer": layer,
                "efficacy": efficacy_score,
                "consistency": consistency_score,
                "fluency": fluency_score,
                "essence": essence_score,
                "score": hmean([
                    efficacy_score,
                    consistency_score,
                    fluency_score,
                    # To be consistent with ROME, we won't use essence for layer sel.
                    # essence_score
                ]),
            }
        else:
            assert dataset == "biosbias"
            correction = load_json(gen_layer_dir / "error_correction_metrics.json")
            accuracy_score = correction["top1_accuracy"]
            fluency_score = correction["fluency"]["mean"]
            row = {
                "layer": layer,
                "top1_accuracy": accuracy_score,
                "consistency": correction["consistency"]["mean"],
                "fluency": fluency_score,
                "score": hmean([accuracy_score, fluency_score])
            }
        rows.append(row)
    return pd.DataFrame(rows)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


MODEL_PLOT_NAMES = {
    "gptj": "GPT-J",
    "gpt2": "GPT2",
    "gpt2-xl": "GPT2-XL",
}

DATASET_PLOT_NAMES = {
    "counterfact": "CounterFact",
    "biosbias": "Bios",
}


def plot_gen_scores_by_layer(model, dataset):
    metrics = load_gen_metrics(model, dataset)

    sns.set(rc={"figure.figsize": (10, 4) if model == "gpt2-xl" else (5, 2)})
    sns.set_style({'font.family':'serif', 'font.serif':['Times New Roman']})

    colors = ["b" if (x < max(metrics.score)) else "darkblue" for x in metrics.score]
    sns.barplot(
        x="layer",
        y="score",
        data=metrics,
        palette=colors,
    )
    plt.title(f"REMEDI Score across Layers ({DATASET_PLOT_NAMES[dataset]})")
    plt.ylabel("Score")
    plt.xlabel("Layer")
    plt.xticks(range(0, len(colors), 4 if model != "gpt2" else 1))
    plt.tight_layout()
    plt.savefig(f"layers_gen_{model}_{dataset}.pdf")
plot_gen_scores_by_layer("gptj", "counterfact")

In [ ]:
plot_gen_scores_by_layer("gptj", "biosbias")

In [ ]:
plot_gen_scores_by_layer("gpt2-xl", "counterfact")

In [ ]:
plot_gen_scores_by_layer("gpt2-xl", "biosbias")

In [ ]:
plot_gen_scores_by_layer("gpt2", "counterfact")

In [ ]:
plot_gen_scores_by_layer("gpt2", "biosbias")

# Classification Tasks: Performance by Layer

In [ ]:
import re

METRICS_FILE_PATTERN = re.compile(r"[a-zA-z_]+_layer_(\d+)_metrics.json")

def get_editor_layer_dir(results_dir):
    return next(iter(layer_dirs(results_dir / "linear")))

def load_cls_metrics(model, dataset):
    results_dir = RESULTS_ROOT / f"post_icml_sweep_cls_{dataset}_{model}"
    assert results_dir.exists()

    editor_layer_dir = get_editor_layer_dir(results_dir)

    metrics_by_entity_layer = {}
    for file in editor_layer_dir.iterdir():
        match = METRICS_FILE_PATTERN.match(file.name)
        if not match:
            continue
        layer = int(match.group(1))
        metrics_by_entity_layer[layer] = load_json(file)

    rows = []
    for missing in range(min(metrics_by_entity_layer)):
        rows.append({
            "layer": missing,
            "f1": 0,
            "mcc": 0,
        })
    
    for layer, metrics in sorted(metrics_by_entity_layer.items(), key=lambda kv: kv[0]):
        if dataset == "counterfact":
            row = {
                "layer": layer,
                "f1": metrics["contextual"]["f1"],
                "mcc": metrics["contextual"]["mcc"],
            }
        else:
            assert dataset == "biosbias"
            row = {
                "layer": layer,
                "f1": metrics["f1"],
                "mcc": metrics["mcc"],
            }
        rows.append(row)
    return pd.DataFrame(rows)

load_cls_metrics("gptj", "counterfact")

In [ ]:
import numpy as np


def plot_cls_scores_by_layer(model, dataset, metric="mcc"):
    metrics = load_cls_metrics(model, dataset)

    sns.set(rc={"figure.figsize": (10, 4) if model == "gpt2-xl" else (5, 2)})
    sns.set_style({'font.family':'serif', 'font.serif':['Times New Roman']})

    colors = ["b" if (x < max(metrics.f1)) else "darkblue" for x in metrics.f1]
    sns.barplot(
        x="layer",
        y=metric,
        data=metrics,
        palette=colors,
    )
    plt.title(f"REMEDI Classification F1 ({DATASET_PLOT_NAMES[dataset]})")
    plt.ylabel(metric.upper())
    plt.xlabel("Layer")
    plt.xticks(range(0, len(colors), 4 if model != "gpt2" else 1))
    plt.yticks(np.linspace(0, 1, 3))
    plt.tight_layout()
    plt.savefig(f"layers_cls_{model}_{dataset}.pdf")

plot_cls_scores_by_layer("gptj", "counterfact", metric="f1")

In [ ]:
plot_cls_scores_by_layer("gptj", "biosbias", metric="f1")

In [ ]:
plot_cls_scores_by_layer("gpt2-xl", "counterfact", metric="f1")

In [ ]:
plot_cls_scores_by_layer("gpt2", "counterfact", metric="f1")

In [ ]:
plot_cls_scores_by_layer("gpt2-xl", "biosbias", metric="f1")

In [ ]:
plot_cls_scores_by_layer("gpt2", "biosbias", metric="f1")

# Entailment Task: Performance by Layer

In [ ]:
def load_ent_metrics(model):
    results_dir = RESULTS_ROOT / f"post_icml_sweep_ent_mcrae_{model}"
    assert results_dir.exists()

    rows = []
    for editor_layer_dir in layer_dirs(results_dir / "linear"):
        if not editor_layer_dir.is_dir():
            continue
        metrics_file = editor_layer_dir / "entailment_metrics.json"
        metrics = load_json(metrics_file)
        row = {
            "layer": int(editor_layer_dir.name),
            **{key: value["mean"] for key, value in metrics.items()}
        }
        rows.append(row)

    return pd.DataFrame(rows)

load_ent_metrics("gptj")

In [ ]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

def plot_ent_scores_by_layer(model, metric="co_corr_post"):
    metrics = load_ent_metrics(model)

    sns.set(rc={"figure.figsize": (10, 4) if model == "gpt2-xl" else (5, 2)})
    sns.set_style({'font.family':'serif', 'font.serif':['Times New Roman']})

    values = getattr(metrics, metric)
    colors = ["b" if (x < max(values)) else "darkblue" for x in values]
    sns.barplot(
        x="layer",
        y=metric,
        data=metrics,
        palette=colors,
    )
    plt.title(f"REMEDI LM-Human Correlation (McRae)")
    plt.ylabel("$r(p_{LM}, p_H)$")
    plt.xlabel("Layer")
    plt.xticks(range(0, len(colors), 4 if model != "gpt2" else 1))
#     plt.yticks(np.linspace(0, .5, 3))
    plt.tight_layout()
    plt.savefig(f"layers_ent_{model}_mcrae.pdf")

plot_ent_scores_by_layer("gptj")

In [ ]:
plot_ent_scores_by_layer("gpt2-xl")

In [ ]:
plot_ent_scores_by_layer("gpt2")

In [ ]:
np.linspace(0, 1, 11)